In [2]:
library("igraph")
library(R.utils)

In [660]:
compute_dist = function(graph, from, to, algorithm){
    
    if(algorithm == c("jaccard"))
        #jaccard returns a 2X2 matrix with diagonal scores giving the distance
        return(similarity(graph, vids = c(from, to), method = c(algorithm))[1,2])
        
    
    else if(algorithm == c("adamicadar"))
        #Invlogweighted accepts only one argument and computes the distance from that to all other vertices
        #which(to == V(new_graph)$name) is used to find the index of to node in the resultant vector
        return(similarity(graph, vids = c(as.character(from)), method = c("invlogweighted"))[which(to == V(graph)$name)])
    
    
    else
        return(length(intersect(neighbors(graph, as.character(from)), neighbors(graph, to))))
    
    
}

In [247]:
file_path = "facebook/"

In [386]:
edge_file <- paste("facebook/facebook_combined.txt" , sep="")
graph <- read_graph(edge_file , format = "edgelist" , directed=FALSE)

In [387]:
V(graph)$name <- V(graph)

In [465]:
neighbors <- c(415, neighbors(graph, as.character(415)))
personal_net = induced_subgraph(graph, vids = as.character(neighbors))
target = V(personal_net)$name[degree(personal_net)==24]
print(target)
writeLines(paste('The number of users in the list is', length(target)))

 [1] 497 579 601 616 619 628 644 659 660 662 663
The number of users in the list is 11


In [651]:
question_17 = function(target, personal_net, algorithm, num_iterations = 10){
    
    node_accuracies <- c()

    for(node in target){

            accuracy <- c()

            for(i in c(1:num_iterations)){

                node_neighbors = neighbors(personal_net, as.character(node))

                deleted_friends <- sample(node_neighbors, length(node_neighbors)*0.25)
                t <- length(deleted_friends) # t is the number of deleted i.e to be recommended friends

                delete_seq <- paste(node, "|", deleted_friends$name, sep = "")

                new_graph <- delete_edges(personal_net, c(delete_seq))

                node_neighbors = neighbors(new_graph, as.character(node))
                node_not_neighbors = difference(V(new_graph), node_neighbors)

                score <- c()
                for(to in node_not_neighbors$name)
                    score <- c(score, compute_dist(new_graph, node, as.character(to), algorithm))
                    #print(score, similarity(new_graph, vids = c(node), method = c("invlogweighted")))

                recommended_friends <- node_not_neighbors[sort(score, decreasing = TRUE, index.return=TRUE)$ix[1:t]]        
                correct_recommendations <- length(intersect(recommended_friends, deleted_friends))

                accuracy <- c(accuracy, correct_recommendations/t)

            }

            printf(paste("\nNode - ", node , " Mean accuracy over 10 trials - ", mean(accuracy)))
            node_accuracies <- c(node_accuracies, mean(accuracy))
    }
    
    return(mean(node_accuracies))
}

In [662]:
acc <- question_17(target, personal_net, "jaccard")
printf(paste("\nAccuracy of Jaccard ", acc))


Node -  497  Mean accuracy over 10 trials -  0.133333333333333
Node -  579  Mean accuracy over 10 trials -  0.816666666666667
Node -  601  Mean accuracy over 10 trials -  0.8
Node -  616  Mean accuracy over 10 trials -  0.683333333333333
Node -  619  Mean accuracy over 10 trials -  0.45
Node -  628  Mean accuracy over 10 trials -  0.833333333333333
Node -  644  Mean accuracy over 10 trials -  0.783333333333333
Node -  659  Mean accuracy over 10 trials -  0.833333333333333
Node -  660  Mean accuracy over 10 trials -  0.833333333333333
Node -  662  Mean accuracy over 10 trials -  0.783333333333333
Node -  663  Mean accuracy over 10 trials -  0.833333333333333
Accuracy of Jaccard  0.707575757575758

In [657]:
acc <- question_17(target, personal_net, "adamicadar")
printf(paste("\nAccuracy of Adamic Adar ", acc))


Node -  497  Mean accuracy over 10 trials -  0.316666666666667
Node -  579  Mean accuracy over 10 trials -  1
Node -  601  Mean accuracy over 10 trials -  0.866666666666667
Node -  616  Mean accuracy over 10 trials -  0.833333333333333
Node -  619  Mean accuracy over 10 trials -  0.466666666666667
Node -  628  Mean accuracy over 10 trials -  0.983333333333333
Node -  644  Mean accuracy over 10 trials -  0.866666666666667
Node -  659  Mean accuracy over 10 trials -  0.983333333333333
Node -  660  Mean accuracy over 10 trials -  1
Node -  662  Mean accuracy over 10 trials -  0.883333333333333
Node -  663  Mean accuracy over 10 trials -  0.983333333333333
Accuracy of Adamic Adar  0.834848484848485

In [664]:
acc <- question_17(target, personal_net, "common_neighbors")
printf(paste("\nAccuracy of Common Neighbors ", acc))


Node -  497  Mean accuracy over 10 trials -  0.2
Node -  579  Mean accuracy over 10 trials -  0.833333333333333
Node -  601  Mean accuracy over 10 trials -  0.8
Node -  616  Mean accuracy over 10 trials -  0.683333333333333
Node -  619  Mean accuracy over 10 trials -  0.433333333333333
Node -  628  Mean accuracy over 10 trials -  0.833333333333333
Node -  644  Mean accuracy over 10 trials -  0.8
Node -  659  Mean accuracy over 10 trials -  0.833333333333333
Node -  660  Mean accuracy over 10 trials -  0.833333333333333
Node -  662  Mean accuracy over 10 trials -  0.783333333333333
Node -  663  Mean accuracy over 10 trials -  0.833333333333333
Accuracy of Common Neighbors  0.715151515151515